In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_iNitroY_Classification_DLNN_CORENup_v2"
outPath = "Results"
foldName = "folds.pickle"

epochs = 100
batch_size = 16
shuffle = True
seed = None

input_data_folder = "Data\\iNitroY-Deep-Dataset"
pos_data_file = "raw-nitrotyrosine-pos.fasta"
neg_data_file = "cdhit70-nitrotyr-neg.fasta"

In [2]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score

import math

from Bio import SeqIO

In [3]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
##################################################################################
##### define all CUSTOM functions
##################################################################################

def one_hot_encode_nt(sequence, char_dict):
    
    seq_encoded = np.zeros((len(sequence),len(char_dict)))
    
    i = 0
    for single_character in sequence:
        if(single_character.upper() in char_dict.keys()):
            seq_encoded[i][char_dict[single_character.upper()]] = 1
            i = i+1
        else:
            raise ValueError('Incorrect character in NT sequence: '+sequence)
    return seq_encoded

In [5]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [6]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

In [7]:
epochs = 200
batch_size = 16
    
##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_CORENup(input_seq_shape = (41, 21),
                 conv_filters_per_layer_1 = 25, kernel_length_1 = 10, conv_strides_1 = 1, ## 1st Convolutional layer parameters
                 max_pool_width_1 = 3, max_pool_stride_1 = 3, ## 1st Maxpool layer parameters
                 lstm_decode_units = 25, ## LSTM layer parameters
                 conv_filters_per_layer_2 = 25,  kernel_length_2 = 5, conv_strides_2 = 1, ## 2nd Convolutional layer parameters
                 max_pool_width_2 = 3, max_pool_stride_2 = 3, ## 2nd Maxpool layer parameters
                 dense_decode_units = 256, ## Dense layer parameters
                 prob = 0.5, learn_rate = 0.0005, 
                 loss = 'binary_crossentropy', metrics = 'accuracy'):
    
    beta = 0.001
    
    ######################################################################################################
    ########  SEQUENCE  ##################################################################################
    ######################################################################################################
    
    input1 = tf.keras.layers.Input(shape=input_seq_shape)

    x1 = tf.keras.layers.Conv1D(conv_filters_per_layer_1, kernel_length_1,
                                strides = conv_strides_1, kernel_regularizer = tf.keras.regularizers.l2(beta), 
                                padding = "same")(input1)
    x1 = tf.keras.layers.Activation('relu')(x1)
    x1 = tf.keras.layers.MaxPool1D(pool_size = max_pool_width_1, strides = max_pool_stride_1)(x1)
    x1 = tf.keras.layers.Dropout(prob)(x1)
    
    x1 = tf.keras.layers.GaussianNoise(stddev=0.1)(x1)

    ## LSTM Path

    x2 = tf.keras.layers.LSTM(lstm_decode_units, return_sequences = True, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta))(x1)
    
    x2 = tf.keras.layers.Dropout(prob)(x2)
    
    x2 = tf.keras.layers.Flatten()(x2)
    
    x2 = tf.keras.layers.GaussianNoise(stddev=0.1)(x2)

    ## Conv Path

    x3 = tf.keras.layers.Conv1D(conv_filters_per_layer_2, kernel_length_2, strides = conv_strides_2, 
                                kernel_regularizer = tf.keras.regularizers.l2(beta), padding = 'same')(x1)
    x3 = tf.keras.layers.Activation('relu')(x3)
    x3 = tf.keras.layers.MaxPooling1D(pool_size = max_pool_width_2, strides = max_pool_stride_2)(x3)
    x3 = tf.keras.layers.Dropout(prob)(x3)
    
    x3 = tf.keras.layers.Flatten()(x3)
    
    x3 = tf.keras.layers.GaussianNoise(stddev=0.1)(x3)
    
    x4 = tf.keras.layers.Concatenate(1)([x2,x3])
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(x4)
    
    y = tf.keras.layers.GaussianNoise(stddev=0.1)(y)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid')(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=input1, outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), loss = loss)

    return model

In [8]:
# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_CORENup(input_seq_shape = (41, 21),
#                  conv_filters_per_layer_1 = 10, kernel_length_1 = 10, conv_strides_1 = 1, ## 1st Convolutional layer parameters
#                  max_pool_width_1 = 3, max_pool_stride_1 = 3, ## 1st Maxpool layer parameters
#                  lstm_decode_units = 10, ## LSTM layer parameters
#                  conv_filters_per_layer_2 = 10,  kernel_length_2 = 5, conv_strides_2 = 1, ## 2nd Convolutional layer parameters
#                  max_pool_width_2 = 3, max_pool_stride_2 = 3, ## 2nd Maxpool layer parameters
#                  dense_decode_units = 32, ## Dense layer parameters
#                  prob = 0.5, learn_rate = 0.0005, 
#                  loss = 'binary_crossentropy', metrics = None):
    
#     beta = 0.001
    
#     ######################################################################################################
#     ########  SEQUENCE  ##################################################################################
#     ######################################################################################################
    
#     input1 = tf.keras.layers.Input(shape=input_seq_shape)

#     x1 = tf.keras.layers.Conv1D(conv_filters_per_layer_1, kernel_length_1,
#                                 strides = conv_strides_1, kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                                 padding = "same")(input1)
#     x1 = tf.keras.layers.Activation('relu')(x1)
#     x1 = tf.keras.layers.MaxPool1D(pool_size = max_pool_width_1, strides = max_pool_stride_1)(x1)
#     x1 = tf.keras.layers.Dropout(prob)(x1)

#     ## LSTM Path

#     x2 = tf.keras.layers.LSTM(lstm_decode_units, return_sequences = True, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta))(x1)
    
#     x2 = tf.keras.layers.Dropout(prob)(x2)
    
#     x2 = tf.keras.layers.Flatten()(x2)

#     ## Conv Path

#     x3 = tf.keras.layers.Conv1D(conv_filters_per_layer_2, kernel_length_2, strides = conv_strides_2, 
#                                 kernel_regularizer = tf.keras.regularizers.l2(beta), padding = 'same')(x1)
#     x3 = tf.keras.layers.Activation('relu')(x3)
#     x3 = tf.keras.layers.MaxPooling1D(pool_size = max_pool_width_2, strides = max_pool_stride_2)(x3)
#     x3 = tf.keras.layers.Dropout(prob)(x3)
    
#     x3 = tf.keras.layers.Flatten()(x3)
    
#     x4 = tf.keras.layers.Concatenate(1)([x2,x3])
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'relu')(x4)
    
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss)

#     return model

In [9]:
# for step in range(10):
#     initial_learning_rate=1e-1
#     decay_steps=10000
#     decay_rate=0.9
#     print(step, ':', initial_learning_rate * decay_rate ** (step / decay_steps))

In [10]:
DLNN_CORENup().summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 41, 21)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 41, 25)       5275        ['input_1[0][0]']                
                                                                                                  
 activation (Activation)        (None, 41, 25)       0           ['conv1d[0][0]']                 
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 13, 25)       0           ['activation[0][0]']             
                                                                                              

# Prepare Dataset

In [11]:
def read_fasta_file(file_path):
    
    openFile = open(file_path)
    fastaSequences = SeqIO.parse(openFile, "fasta")

    name_list = []
    seq_list = []

    for fasta in fastaSequences: 
        name_list.append(fasta.id)
        seq_list.append(str(fasta.seq))

    openFile.close()
    
    return name_list, seq_list

In [12]:
##################################################################################
##### read positive and negative files
##################################################################################

pos_file_path = os.path.join(input_data_folder, pos_data_file)
_, pos_seq_list = read_fasta_file(pos_file_path)

neg_file_path = os.path.join(input_data_folder, neg_data_file)
_, neg_seq_list = read_fasta_file(neg_file_path)

pos_seq_list = [val.replace('X', '-') for val in pos_seq_list]
neg_seq_list = [val.replace('X', '-') for val in neg_seq_list]

# remove duplicates in data
pos_seq_list = list(set(pos_seq_list))
neg_seq_list = list(set(neg_seq_list))

all_seq_list = pos_seq_list + neg_seq_list

all_seq_label_list = ([1] * len(pos_seq_list)) + ([0] * len(neg_seq_list))

##################################################################################
##### Create dictionary of all characters in the NT sequence 
##################################################################################
all_char_set = set({})
for val in [set(val) for val in all_seq_list]:
    all_char_set = all_char_set.union(val)
all_char_list = list(all_char_set)
all_char_list.sort()
all_char_dict = {}
for i in range(len(all_char_list)):
    all_char_dict[all_char_list[i]] = i
    
##################################################################################
##### Create OHE of all sequences
##################################################################################
all_seq_OHE_list = [one_hot_encode_nt(val, all_char_dict)
                    for val in all_seq_list]

##################################################################################
##### Create numpy array of features and sequences
##################################################################################

## create the features and labels datasets for the training
features = np.array(all_seq_OHE_list)
labels = np.array(all_seq_label_list)
labels = labels.reshape((labels.shape[0], 1))

##################################################################################
##### Divide into Train/Independent datasets
##################################################################################

train_features, indpe_features, train_labels, indpe_labels = train_test_split(features, labels, 
                                                                              stratify=labels, test_size=0.3, 
                                                                              random_state=seed, shuffle=shuffle)

##################################################################################
##### Generate Folds from training dataset, and store to file
##################################################################################

## Generate the k-fold dataset
folds = build_kfold(train_features, train_labels, k=n_fold, shuffle=shuffle, seed=seed)

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

## Write the independent test dataset to file
pickle.dump([indpe_features, indpe_labels], open(os.path.join(foldPath, 'independent_dataset.pickle'), "wb"))

##################################################################################

input_seq_shape = features[0].shape

# Training

In [13]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Train/Test model on all folds, generate evaluations
##################################################################################

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

i = -1
for fold in folds:
    i += 1
    
    print("\nTrain/Test model on Fold #"+str(i)+".")
    
    model = DLNN_CORENup(input_seq_shape = input_seq_shape)
    
    ## Define the model callbacks for early stopping and saving the model. Then train model
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    modelCallbacks = [
        tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                           monitor = 'val_loss', verbose = 1, save_best_only = True, 
                                           save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
    ]
    
    # adding random shuffling of the dataset for training purpose
    index_arr = np.arange(fold["X_train"].shape[0])
    index_arr = np.random.permutation(index_arr)
    
    model.fit(x = fold["X_train"][index_arr], y = fold["y_train"][index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
              callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))
    
    model = tf.keras.models.load_model(current_model_path)
    
    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    if(conf[0][0]+conf[1][0]):
        sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
    else:
        sens = 0.0
    if(conf[1][1]+conf[0][1]):
        spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
    else:
        spec = 0.0
    if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
        mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
    else:
        mcc= 0.0
    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    if(conf[0][0]+conf[1][0]):
        sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
    else:
        sens = 0.0
    if(conf[1][1]+conf[0][1]):
        spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
    else:
        spec = 0.0
    if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
        mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
    else:
        mcc= 0.0
    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)


Train/Test model on Fold #0.
Epoch 1/200
24/24 [==============================] - ETA: 0s - loss: 0.9959 - accuracy: 0.7181
Epoch 1: val_loss improved from inf to 0.95466, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold0.hdf5
24/24 [==============================] - 5s 42ms/step - loss: 0.9959 - accuracy: 0.7181 - val_loss: 0.9547 - val_accuracy: 0.7474
Epoch 2/200
24/24 [==============================] - ETA: 0s - loss: 0.9570 - accuracy: 0.7527
Epoch 2: val_loss improved from 0.95466 to 0.93822, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold0.hdf5
24/24 [==============================] - 0s 13ms/step - loss: 0.9570 - accuracy: 0.7527 - val_loss: 0.9382 - val_accuracy: 0.7474
Epoch 3/200
24/24 [==============================] - ETA: 0s - loss: 0.9234 - accuracy: 0.7606
Epoch 3: val_loss improved from 0.93822 to 0.91475, saving model to Results\NT_Site_iNitroY_Classification_DLNN_COR

24/24 [==============================] - 0s 14ms/step - loss: 0.4393 - accuracy: 0.9043 - val_loss: 0.5190 - val_accuracy: 0.8316
Epoch 23/200
19/24 [======================>.......] - ETA: 0s - loss: 0.4121 - accuracy: 0.9276
Epoch 23: val_loss did not improve from 0.51899
24/24 [==============================] - 0s 11ms/step - loss: 0.4316 - accuracy: 0.9176 - val_loss: 0.5274 - val_accuracy: 0.8316
Epoch 24/200
19/24 [======================>.......] - ETA: 0s - loss: 0.3699 - accuracy: 0.9375
Epoch 24: val_loss did not improve from 0.51899
24/24 [==============================] - 0s 11ms/step - loss: 0.3785 - accuracy: 0.9335 - val_loss: 0.5217 - val_accuracy: 0.8316
Epoch 25/200
19/24 [======================>.......] - ETA: 0s - loss: 0.3909 - accuracy: 0.9408
Epoch 25: val_loss improved from 0.51899 to 0.47495, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\bestModel-fold0.hdf5
24/24 [==============================] - 0s 13ms/step - loss: 0.3762

Epoch 50/200
19/24 [======================>.......] - ETA: 0s - loss: 0.2044 - accuracy: 0.9704
Epoch 50: val_loss did not improve from 0.39855
24/24 [==============================] - 0s 11ms/step - loss: 0.2038 - accuracy: 0.9681 - val_loss: 0.4440 - val_accuracy: 0.9158
Epoch 51/200
19/24 [======================>.......] - ETA: 0s - loss: 0.1891 - accuracy: 0.9836
Epoch 51: val_loss did not improve from 0.39855
24/24 [==============================] - 0s 11ms/step - loss: 0.1960 - accuracy: 0.9787 - val_loss: 0.4275 - val_accuracy: 0.9053
Epoch 52/200
19/24 [======================>.......] - ETA: 0s - loss: 0.1929 - accuracy: 0.9737
Epoch 52: val_loss did not improve from 0.39855
24/24 [==============================] - 0s 12ms/step - loss: 0.1893 - accuracy: 0.9734 - val_loss: 0.4016 - val_accuracy: 0.9053
Epoch 53/200
20/24 [========================>.....] - ETA: 0s - loss: 0.2029 - accuracy: 0.9531
Epoch 53: val_loss did not improve from 0.39855
24/24 [===========================

Epoch 79/200
19/24 [======================>.......] - ETA: 0s - loss: 0.1460 - accuracy: 0.9803
Epoch 79: val_loss did not improve from 0.36264
24/24 [==============================] - 0s 12ms/step - loss: 0.1543 - accuracy: 0.9787 - val_loss: 0.4424 - val_accuracy: 0.9263
Epoch 80/200
24/24 [==============================] - ETA: 0s - loss: 0.1236 - accuracy: 0.9867
Epoch 80: val_loss did not improve from 0.36264
24/24 [==============================] - 0s 12ms/step - loss: 0.1236 - accuracy: 0.9867 - val_loss: 0.3816 - val_accuracy: 0.9053
Epoch 81/200
24/24 [==============================] - ETA: 0s - loss: 0.1255 - accuracy: 0.9920
Epoch 81: val_loss did not improve from 0.36264
24/24 [==============================] - 0s 12ms/step - loss: 0.1255 - accuracy: 0.9920 - val_loss: 0.3891 - val_accuracy: 0.8947
Epoch 82/200
24/24 [==============================] - ETA: 0s - loss: 0.1035 - accuracy: 0.9894
Epoch 82: val_loss did not improve from 0.36264
24/24 [===========================

24/24 [==============================] - 0s 11ms/step - loss: 0.1129 - accuracy: 0.9814 - val_loss: 0.4192 - val_accuracy: 0.9263
Epoch 109/200
19/24 [======================>.......] - ETA: 0s - loss: 0.1020 - accuracy: 0.9901
Epoch 109: val_loss did not improve from 0.35457
24/24 [==============================] - 0s 11ms/step - loss: 0.0991 - accuracy: 0.9894 - val_loss: 0.4170 - val_accuracy: 0.9158
Epoch 110/200
19/24 [======================>.......] - ETA: 0s - loss: 0.1110 - accuracy: 0.9836
Epoch 110: val_loss did not improve from 0.35457
24/24 [==============================] - 0s 11ms/step - loss: 0.1080 - accuracy: 0.9840 - val_loss: 0.4719 - val_accuracy: 0.9263
Epoch 111/200
24/24 [==============================] - ETA: 0s - loss: 0.1036 - accuracy: 0.9920
Epoch 111: val_loss did not improve from 0.35457
24/24 [==============================] - 0s 12ms/step - loss: 0.1036 - accuracy: 0.9920 - val_loss: 0.4507 - val_accuracy: 0.9263
Epoch 112/200
22/24 [=====================

...

23/24 [===========================>..] - ETA: 0s - loss: 0.1824 - accuracy: 0.9810
Epoch 62: val_loss did not improve from 0.44554
24/24 [==============================] - 0s 12ms/step - loss: 0.1814 - accuracy: 0.9814 - val_loss: 0.5029 - val_accuracy: 0.9149
Epoch 63/200
24/24 [==============================] - ETA: 0s - loss: 0.1590 - accuracy: 0.9841
Epoch 63: val_loss did not improve from 0.44554
24/24 [==============================] - 0s 12ms/step - loss: 0.1590 - accuracy: 0.9841 - val_loss: 0.5475 - val_accuracy: 0.9255
Epoch 64/200
19/24 [======================>.......] - ETA: 0s - loss: 0.1555 - accuracy: 0.9803
Epoch 64: val_loss did not improve from 0.44554
24/24 [==============================] - 0s 12ms/step - loss: 0.1584 - accuracy: 0.9788 - val_loss: 0.5255 - val_accuracy: 0.9255
Epoch 65/200
19/24 [======================>.......] - ETA: 0s - loss: 0.1633 - accuracy: 0.9836
Epoch 65: val_loss did not improve from 0.44554
24/24 [==============================] - 0s 11m

Epoch 92/200
19/24 [======================>.......] - ETA: 0s - loss: 0.1070 - accuracy: 0.9934
Epoch 92: val_loss did not improve from 0.42903
24/24 [==============================] - 0s 11ms/step - loss: 0.1046 - accuracy: 0.9947 - val_loss: 0.5671 - val_accuracy: 0.9149
Epoch 93/200
19/24 [======================>.......] - ETA: 0s - loss: 0.1190 - accuracy: 0.9868
Epoch 93: val_loss did not improve from 0.42903
24/24 [==============================] - 0s 11ms/step - loss: 0.1144 - accuracy: 0.9894 - val_loss: 0.6328 - val_accuracy: 0.9149
Epoch 94/200
19/24 [======================>.......] - ETA: 0s - loss: 0.1074 - accuracy: 0.9901
Epoch 94: val_loss did not improve from 0.42903
24/24 [==============================] - 0s 11ms/step - loss: 0.1061 - accuracy: 0.9894 - val_loss: 0.5400 - val_accuracy: 0.9149
Epoch 95/200
23/24 [===========================>..] - ETA: 0s - loss: 0.1048 - accuracy: 0.9891
Epoch 95: val_loss did not improve from 0.42903
24/24 [===========================

Epoch 122/200
24/24 [==============================] - ETA: 0s - loss: 0.0953 - accuracy: 0.9894
Epoch 122: val_loss did not improve from 0.42903
24/24 [==============================] - 0s 11ms/step - loss: 0.0953 - accuracy: 0.9894 - val_loss: 0.6030 - val_accuracy: 0.9149
Epoch 123/200
24/24 [==============================] - ETA: 0s - loss: 0.0900 - accuracy: 0.9894
Epoch 123: val_loss did not improve from 0.42903
24/24 [==============================] - 0s 12ms/step - loss: 0.0900 - accuracy: 0.9894 - val_loss: 0.5165 - val_accuracy: 0.9149
Epoch 124/200
22/24 [==========================>...] - ETA: 0s - loss: 0.0930 - accuracy: 0.9915
Epoch 124: val_loss did not improve from 0.42903
24/24 [==============================] - 0s 12ms/step - loss: 0.0927 - accuracy: 0.9920 - val_loss: 0.5636 - val_accuracy: 0.9149
Epoch 125/200
24/24 [==============================] - ETA: 0s - loss: 0.0923 - accuracy: 0.9920
Epoch 125: val_loss did not improve from 0.42903
24/24 [===================

24/24 [==============================] - ETA: 0s - loss: 0.0739 - accuracy: 0.9947
Epoch 151: val_loss did not improve from 0.42316
24/24 [==============================] - 0s 12ms/step - loss: 0.0739 - accuracy: 0.9947 - val_loss: 0.6609 - val_accuracy: 0.9255
Epoch 152/200
24/24 [==============================] - ETA: 0s - loss: 0.0740 - accuracy: 0.9947
Epoch 152: val_loss did not improve from 0.42316
24/24 [==============================] - 0s 11ms/step - loss: 0.0740 - accuracy: 0.9947 - val_loss: 0.6455 - val_accuracy: 0.9149
Epoch 153/200
19/24 [======================>.......] - ETA: 0s - loss: 0.0711 - accuracy: 0.9934
Epoch 153: val_loss did not improve from 0.42316
24/24 [==============================] - 0s 11ms/step - loss: 0.0804 - accuracy: 0.9894 - val_loss: 0.5531 - val_accuracy: 0.9149
Epoch 154/200
19/24 [======================>.......] - ETA: 0s - loss: 0.0697 - accuracy: 0.9934
Epoch 154: val_loss did not improve from 0.42316
24/24 [==============================] -

Epoch 181/200
19/24 [======================>.......] - ETA: 0s - loss: 0.0689 - accuracy: 0.9934
Epoch 181: val_loss did not improve from 0.42316
24/24 [==============================] - 0s 11ms/step - loss: 0.0664 - accuracy: 0.9947 - val_loss: 0.6455 - val_accuracy: 0.9255
Epoch 182/200
22/24 [==========================>...] - ETA: 0s - loss: 0.0836 - accuracy: 0.9886
Epoch 182: val_loss did not improve from 0.42316
24/24 [==============================] - 0s 12ms/step - loss: 0.0822 - accuracy: 0.9894 - val_loss: 0.5693 - val_accuracy: 0.9255
Epoch 183/200
19/24 [======================>.......] - ETA: 0s - loss: 0.0724 - accuracy: 0.9967
Epoch 183: val_loss did not improve from 0.42316
24/24 [==============================] - 0s 11ms/step - loss: 0.0757 - accuracy: 0.9947 - val_loss: 0.5345 - val_accuracy: 0.9255
Epoch 184/200
19/24 [======================>.......] - ETA: 0s - loss: 0.0628 - accuracy: 0.9967
Epoch 184: val_loss did not improve from 0.42316
24/24 [===================

## k-fold Training evaluation

In [14]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.927839,0.893434,0.908252,0.938269,0.893434,0.800014
Train,0.994695,1.000000,0.999447,0.993197,1.000000,0.985700


In [15]:
# batch 16
# 	Accuracy	Precision	AUC	Sensitivity	Specificity	MCC
# Train_Test						
# Test	0.943310	0.923092	0.965850	0.957041	0.923092	0.881812
# Train	0.997427	0.995646	0.999898	0.998589	0.995646	0.994615

In [16]:
evaluations_df[evaluations_df["Train_Test"] == "Test"]

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
1,0,Test,0.915789,0.863636,"[0.0, 0.041666666666666664, 0.75, 0.75, 0.7916...","[0.0, 0.0, 0.0, 0.04225352112676056, 0.0422535...","[1.9992676, 0.99926764, 0.9817755, 0.54409903,...",0.945423,0.931507,0.863636,0.771940
3,1,Test,0.968085,0.916667,"[0.0, 0.043478260869565216, 0.6086956521739131...","[0.0, 0.0, 0.0, 0.014084507042253521, 0.014084...","[1.998269, 0.99826896, 0.986548, 0.984014, 0.8...",0.956522,0.985714,0.916667,0.915275
5,2,Test,0.946809,0.909091,"[0.0, 0.043478260869565216, 0.7391304347826086...","[0.0, 0.0, 0.0, 0.014084507042253521, 0.014084...","[1.9986198, 0.9986198, 0.7584738, 0.70890576, ...",0.911206,0.958333,0.909091,0.854311
7,3,Test,0.882979,0.833333,"[0.0, 0.043478260869565216, 0.0869565217391304...","[0.0, 0.0, 0.0, 0.014084507042253521, 0.014084...","[1.993515, 0.993515, 0.99283034, 0.9925287, 0....",0.824250,0.894737,0.833333,0.666382
9,4,Test,0.925532,0.944444,"[0.0, 0.043478260869565216, 0.6956521739130435...","[0.0, 0.0, 0.0, 0.014084507042253521, 0.014084...","[1.9993758, 0.99937576, 0.832517, 0.736401, 0....",0.903858,0.921053,0.944444,0.792165


# Independent data

## Using k-fold Models

### Performance of each k-fold model

In [17]:
## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    label_pred = pred2label(y_pred)

    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(indpe_labels, label_pred)
    prec = precision_score(indpe_labels,label_pred)

    conf = confusion_matrix(indpe_labels, label_pred)
    if(conf[0][0]+conf[1][0]):
        sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
    else:
        sens = 0.0
    if(conf[1][1]+conf[0][1]):
        spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
    else:
        spec = 0.0
    if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
        mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
    else:
        mcc= 0.0
    fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
    auc = roc_auc_score(indpe_labels, y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Independent")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.89802,0.863203,0.843395,0.908845,0.863203,0.715715


In [18]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Independent,0.910891,0.847826,"[0.0, 0.02, 0.76, 0.76, 0.78, 0.78, 0.8, 0.8, ...","[0.0, 0.0, 0.0, 0.02631578947368421, 0.0263157...","[1.9989231, 0.998923, 0.9648313, 0.94721776, 0...",0.884474,0.929487,0.847826,0.755319
1,1,Independent,0.886139,0.787234,"[0.0, 0.02, 0.62, 0.62, 0.7, 0.7, 0.72, 0.72, ...","[0.0, 0.0, 0.0, 0.006578947368421052, 0.006578...","[1.9987915, 0.9987914, 0.9806472, 0.9795981, 0...",0.851711,0.916129,0.787234,0.688633
2,2,Independent,0.910891,0.880952,"[0.0, 0.02, 0.7, 0.7, 0.72, 0.72, 0.74, 0.74, ...","[0.0, 0.0, 0.0, 0.006578947368421052, 0.006578...","[1.9991064, 0.99910647, 0.9734899, 0.95179695,...",0.860526,0.918750,0.880952,0.751980
3,3,Independent,0.871287,0.900000,"[0.0, 0.02, 0.04, 0.04, 0.34, 0.34, 0.52, 0.52...","[0.0, 0.0, 0.0, 0.006578947368421052, 0.006578...","[1.9936438, 0.99364376, 0.98176193, 0.9809605,...",0.796053,0.866279,0.900000,0.631401
4,4,Independent,0.910891,0.900000,"[0.0, 0.02, 0.28, 0.28, 0.62, 0.62, 0.66, 0.66...","[0.0, 0.0, 0.0, 0.006578947368421052, 0.006578...","[1.999385, 0.999385, 0.996316, 0.9962303, 0.98...",0.824211,0.913580,0.900000,0.751244


### Mean score with k-fold models

In [19]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    total_pred += y_pred
    all_preds.append(y_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
if(conf[0][0]+conf[1][0]):
    sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
else:
    sens = 0.0
if(conf[1][1]+conf[0][1]):
    spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
else:
    spec = 0.0
if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
    mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
else:
    mcc= 0.0
fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.920792,0.886364,0.874737,0.93038,0.886364,0.781149


### Voting score with k-fold models

In [20]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    vote_pred = pred2label(y_pred)
    total_pred += vote_pred
    all_preds.append(vote_pred)
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
if(conf[0][0]+conf[1][0]):
    sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
else:
    sens = 0.0
if(conf[1][1]+conf[0][1]):
    spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
else:
    spec = 0.0
if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
    mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
else:
    mcc= 0.0
fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.930693,0.928571,0.874079,0.93125,0.928571,0.808511


## Using New Model

Train one model on full data from training. Predict and evaluate on Independent data.

In [21]:
model = DLNN_CORENup(input_seq_shape = input_seq_shape)
    
## Define the model callbacks for early stopping and saving the model. Then train model
current_model_path = os.path.join(modelPath, "_fullModel.hdf5")
modelCallbacks = [
    tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                       monitor = 'val_loss', verbose = 1, save_best_only = True, 
                                       save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
]

# adding random shuffling of the dataset for training purpose
index_arr = np.arange(train_features.shape[0])
index_arr = np.random.permutation(index_arr)

model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
          callbacks = modelCallbacks, validation_data = (indpe_features, indpe_labels))
# model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
#           callbacks = modelCallbacks, validation_split = 0.2)

model = tf.keras.models.load_model(current_model_path)

Epoch 1/200
30/30 [==============================] - ETA: 0s - loss: 1.0263 - accuracy: 0.6815
Epoch 1: val_loss improved from inf to 0.93522, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\_fullModel.hdf5
30/30 [==============================] - 3s 32ms/step - loss: 1.0263 - accuracy: 0.6815 - val_loss: 0.9352 - val_accuracy: 0.7525
Epoch 2/200
25/30 [========================>.....] - ETA: 0s - loss: 0.9624 - accuracy: 0.7400
Epoch 2: val_loss improved from 0.93522 to 0.91562, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\_fullModel.hdf5
30/30 [==============================] - 0s 14ms/step - loss: 0.9551 - accuracy: 0.7452 - val_loss: 0.9156 - val_accuracy: 0.7525
Epoch 3/200
25/30 [========================>.....] - ETA: 0s - loss: 0.9330 - accuracy: 0.7400
Epoch 3: val_loss improved from 0.91562 to 0.88848, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\_fullModel.hdf5
30/

Epoch 23/200
25/30 [========================>.....] - ETA: 0s - loss: 0.3729 - accuracy: 0.9175
Epoch 23: val_loss improved from 0.54238 to 0.50634, saving model to Results\NT_Site_iNitroY_Classification_DLNN_CORENup_v2\5fold\models\_fullModel.hdf5
30/30 [==============================] - 0s 14ms/step - loss: 0.3781 - accuracy: 0.9172 - val_loss: 0.5063 - val_accuracy: 0.8960
Epoch 24/200
25/30 [========================>.....] - ETA: 0s - loss: 0.3338 - accuracy: 0.9350
Epoch 24: val_loss did not improve from 0.50634
30/30 [==============================] - 0s 11ms/step - loss: 0.3407 - accuracy: 0.9257 - val_loss: 0.5343 - val_accuracy: 0.8911
Epoch 25/200
25/30 [========================>.....] - ETA: 0s - loss: 0.3506 - accuracy: 0.9175
Epoch 25: val_loss did not improve from 0.50634
30/30 [==============================] - 0s 12ms/step - loss: 0.3394 - accuracy: 0.9257 - val_loss: 0.5522 - val_accuracy: 0.8812
Epoch 26/200
29/30 [============================>.] - ETA: 0s - loss: 0.2

30/30 [==============================] - 0s 12ms/step - loss: 0.1778 - accuracy: 0.9682 - val_loss: 0.5113 - val_accuracy: 0.9109
Epoch 52/200
25/30 [========================>.....] - ETA: 0s - loss: 0.1637 - accuracy: 0.9800
Epoch 52: val_loss did not improve from 0.47054
30/30 [==============================] - 0s 12ms/step - loss: 0.1634 - accuracy: 0.9766 - val_loss: 0.5124 - val_accuracy: 0.9109
Epoch 53/200
28/30 [===========================>..] - ETA: 0s - loss: 0.1494 - accuracy: 0.9844
Epoch 53: val_loss did not improve from 0.47054
30/30 [==============================] - 0s 12ms/step - loss: 0.1534 - accuracy: 0.9830 - val_loss: 0.5313 - val_accuracy: 0.9257
Epoch 54/200
29/30 [============================>.] - ETA: 0s - loss: 0.1790 - accuracy: 0.9741
Epoch 54: val_loss did not improve from 0.47054
30/30 [==============================] - 0s 13ms/step - loss: 0.1779 - accuracy: 0.9745 - val_loss: 0.4852 - val_accuracy: 0.9158
Epoch 55/200
25/30 [========================>...

30/30 [==============================] - 0s 12ms/step - loss: 0.1136 - accuracy: 0.9851 - val_loss: 0.5089 - val_accuracy: 0.9109
Epoch 82/200
25/30 [========================>.....] - ETA: 0s - loss: 0.1140 - accuracy: 0.9850
Epoch 82: val_loss did not improve from 0.47054
30/30 [==============================] - 0s 11ms/step - loss: 0.1099 - accuracy: 0.9851 - val_loss: 0.5377 - val_accuracy: 0.9257
Epoch 83/200
25/30 [========================>.....] - ETA: 0s - loss: 0.1067 - accuracy: 0.9875
Epoch 83: val_loss did not improve from 0.47054
30/30 [==============================] - 0s 11ms/step - loss: 0.1089 - accuracy: 0.9851 - val_loss: 0.5358 - val_accuracy: 0.9257
Epoch 84/200
25/30 [========================>.....] - ETA: 0s - loss: 0.0945 - accuracy: 0.9900
Epoch 84: val_loss did not improve from 0.47054
30/30 [==============================] - 0s 11ms/step - loss: 0.0982 - accuracy: 0.9894 - val_loss: 0.5392 - val_accuracy: 0.9257
Epoch 85/200
25/30 [========================>...

25/30 [========================>.....] - ETA: 0s - loss: 0.0748 - accuracy: 0.9950
Epoch 111: val_loss did not improve from 0.47054
30/30 [==============================] - 0s 12ms/step - loss: 0.0741 - accuracy: 0.9958 - val_loss: 0.5758 - val_accuracy: 0.9307
Epoch 112/200
25/30 [========================>.....] - ETA: 0s - loss: 0.0950 - accuracy: 0.9925
Epoch 112: val_loss did not improve from 0.47054
30/30 [==============================] - 0s 12ms/step - loss: 0.0953 - accuracy: 0.9915 - val_loss: 0.5676 - val_accuracy: 0.9356
Epoch 113/200
25/30 [========================>.....] - ETA: 0s - loss: 0.1047 - accuracy: 0.9850
Epoch 113: val_loss did not improve from 0.47054
30/30 [==============================] - 0s 11ms/step - loss: 0.1089 - accuracy: 0.9851 - val_loss: 0.5343 - val_accuracy: 0.9307
Epoch 114/200
25/30 [========================>.....] - ETA: 0s - loss: 0.0904 - accuracy: 0.9850
Epoch 114: val_loss did not improve from 0.47054
30/30 [==============================] -

30/30 [==============================] - 0s 12ms/step - loss: 0.0863 - accuracy: 0.9915 - val_loss: 0.5043 - val_accuracy: 0.9257
Epoch 141/200
30/30 [==============================] - ETA: 0s - loss: 0.0922 - accuracy: 0.9915
Epoch 141: val_loss did not improve from 0.45972
30/30 [==============================] - 0s 12ms/step - loss: 0.0922 - accuracy: 0.9915 - val_loss: 0.4607 - val_accuracy: 0.9208
Epoch 142/200
30/30 [==============================] - ETA: 0s - loss: 0.0779 - accuracy: 0.9936
Epoch 142: val_loss did not improve from 0.45972
30/30 [==============================] - 0s 12ms/step - loss: 0.0779 - accuracy: 0.9936 - val_loss: 0.4992 - val_accuracy: 0.9257
Epoch 143/200
25/30 [========================>.....] - ETA: 0s - loss: 0.0817 - accuracy: 0.9900
Epoch 143: val_loss did not improve from 0.45972
30/30 [==============================] - 0s 11ms/step - loss: 0.0789 - accuracy: 0.9915 - val_loss: 0.5236 - val_accuracy: 0.9356
Epoch 144/200
25/30 [=====================

Epoch 170/200
28/30 [===========================>..] - ETA: 0s - loss: 0.0796 - accuracy: 0.9888
Epoch 170: val_loss did not improve from 0.45911
30/30 [==============================] - 0s 13ms/step - loss: 0.0784 - accuracy: 0.9894 - val_loss: 0.4919 - val_accuracy: 0.9257
Epoch 171/200
30/30 [==============================] - ETA: 0s - loss: 0.0734 - accuracy: 0.9915
Epoch 171: val_loss did not improve from 0.45911
30/30 [==============================] - 0s 12ms/step - loss: 0.0734 - accuracy: 0.9915 - val_loss: 0.5240 - val_accuracy: 0.9208
Epoch 172/200
25/30 [========================>.....] - ETA: 0s - loss: 0.1073 - accuracy: 0.9800
Epoch 172: val_loss did not improve from 0.45911
30/30 [==============================] - 0s 12ms/step - loss: 0.1071 - accuracy: 0.9809 - val_loss: 0.4775 - val_accuracy: 0.9109
Epoch 173/200
30/30 [==============================] - ETA: 0s - loss: 0.0666 - accuracy: 1.0000
Epoch 173: val_loss did not improve from 0.45911
30/30 [===================

Epoch 200/200
29/30 [============================>.] - ETA: 0s - loss: 0.0712 - accuracy: 0.9935
Epoch 200: val_loss did not improve from 0.45911
30/30 [==============================] - 0s 12ms/step - loss: 0.0720 - accuracy: 0.9936 - val_loss: 0.6323 - val_accuracy: 0.9158


In [22]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
if(conf[0][0]+conf[1][0]):
    sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
else:
    sens = 0.0
if(conf[1][1]+conf[0][1]):
    spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
else:
    spec = 0.0
if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
    mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
else:
    mcc= 0.0
fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.920792,0.869565,0.881447,0.935897,0.869565,0.782672


In [23]:
# out_fasta_file_name = '.'.join((training_data_file.split('.')[0], 'fasta'))
# out_fasta_file_path = os.path.join(input_data_folder, out_fasta_file_name)

# count = 0
# list_seqs = list(train_data['Sequence'])

# with open(out_fasta_file_path, "w") as out_file_obj:
#     for strLine in list_seqs:
        
#         #Output the header
#         out_file_obj.write(">" + str(count+1) + "\n")
#         out_file_obj.write(strLine + "\n")
        
#         count += 1